In [ ]:
from main import CRLoader as Loader, FExtractorB as FExtractor, TrainerB as Trainer, TesterB as Tester

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from UCroma import PretrainedCROMA
import copy
import pandas as pd
import os
import pickle
    
sample_size = 120
batch_size = 16

Croma = PretrainedCROMA(pretrained_path='CR.pt', size='base', modality='both', image_resolution=sample_size)
num_features = Croma(SAR_images=torch.randn(1, 2, sample_size, sample_size), 
                     optical_images=torch.randn(1, 12, sample_size, sample_size))['joint_GAP'].shape[1]
head = nn.Sequential(nn.Linear(num_features, 1000), nn.ReLU(), nn.Linear(1000, 1000), nn.ReLU(), nn.Linear(1000, 2))

In [ ]:
it = 10
metrics = []

data = {}
for j in range(1, 5):
    root_dir = f"/home/stagiaire/D/D/patchs/{j}n"  
    for folder in os.listdir(root_dir):
        root_folder = os.path.join(root_dir, folder)
        files = os.listdir(root_folder)
        for file in files:
            n = file.split('.')[0]
            data[n] = []
global_list = pd.DataFrame(list(data.items()), columns=['Id', 'List'])
global_list = global_list.set_index('Id')

processor = Loader(opt_root_dir=f"/home/stagiaire/D/D/patchs/{3}n", sar_root_dir=f"/home/stagiaire/D/D/patchs/{3}Rn", num_folds=1)
processor.load_data(sample_size=sample_size, batch_size=batch_size)
loader = processor.loaders[0]
feature_extractor = FExtractor(dataloader=loader, use_8_bit=True)
features, labels, ids = feature_extractor.extract_features()
dataset = TensorDataset(torch.from_numpy(features), torch.from_numpy(labels), torch.from_numpy(ids))
f_test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

for i in range(it):

    processor = Loader(opt_root_dir=f"/home/stagiaire/D/D/patchs/{412}n", sar_root_dir=f"/home/stagiaire/D/D/patchs/{412}Rn", num_folds=1)
    processor.load_data(sample_size=sample_size, batch_size=batch_size)
    loader = processor.loaders[0]
    feature_extractor = FExtractor(dataloader=loader, use_8_bit=True)
    features, labels, ids = feature_extractor.extract_features()
    dataset = TensorDataset(torch.from_numpy(features), torch.from_numpy(labels), torch.from_numpy(ids))
    f_train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    head_i = copy.deepcopy(head)
        
    trainer = Trainer(head_i, f_train_loader)
    trainer.train(num_epochs=40)
        
    tester = Tester(head_i, f_test_loader, processor.combined_dataset)
    tester.evaluate(global_list)
    
    metrics.append(tester.report)
                
with open(f"/home/stagiaire/D/R/metrics/MV{it}3.pkl", 'wb') as f:
    pickle.dump(global_list, f)

In [ ]:
from R import RGenerator

report = RGenerator(metrics).report()